In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()

24/11/23 00:12:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/23 00:12:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/23 00:12:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/23 00:12:58 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/23 00:12:58 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.functions import col, current_timestamp

In [3]:
results_schema = StructType([
                        StructField('resultId', IntegerType()),
                        StructField('raceId', IntegerType()),
                        StructField('driverId', IntegerType()),
                        StructField('constructorId', IntegerType()),
                        StructField('number', IntegerType()),
                        StructField('grid', IntegerType()),
                        StructField('position', IntegerType()),
                        StructField('positionText', StringType()),
                        StructField('positionOrder', IntegerType()),
                        StructField('points', FloatType()),
                        StructField('laps', IntegerType()),
                        StructField('time', StringType()),
                        StructField('milliseconds', IntegerType()),
                        StructField('fastestLap',IntegerType()),
                        StructField('rank',IntegerType()),
                        StructField('fastestLapTime',StringType()),
                        StructField('fastestLapSpeed', FloatType()),
                        StructField('statusId',StringType())
                            ])

In [4]:
results_df= spark.read.schema(results_schema).format('json').load('source/results.json').drop('statusId')


In [17]:
results_withrenamed_df = results_df.withColumnRenamed('resultId','result_id')\
                            .withColumnRenamed('raceId','race_id')\
                            .withColumnRenamed('driverId','driver_id')\
                            .withColumnRenamed('constructorId','constructor_id')\
                            .withColumnRenamed('positionText','position_text')\
                            .withColumnRenamed('positionOrder','position_order')\
                            .withColumnRenamed('fastestLap','fastest_lap')\
                            .withColumnRenamed('rank','ranks')\
                            .withColumnRenamed('fastestLapTime','fastest_lap_time')\
                            .withColumnRenamed('fastestLapSpeed','fastest_lap_speed')\
                            .withColumn('ingestion_date', current_timestamp())

In [18]:
results_withrenamed_df.write.partitionBy('race_id').option('header',True).mode('overwrite').format('csv').save('tbls/results')


In [24]:
results_withrenamed_df.coalesce(1).write.option('header',True).mode('overwrite').format('csv').save('tbls/result')


In [5]:
df = spark.read.option('header',True).option('inferSchema',True).csv('tbls/result/result.csv')


In [16]:
outs = []
for i in df.select('fastest_lap_time').collect():
    outs.append(len(i[0]))
print(max(outs))

8


In [21]:
df.printSchema()

root
 |-- result_id: integer (nullable = true)
 |-- race_id: integer (nullable = true)
 |-- driver_id: integer (nullable = true)
 |-- constructor_id: integer (nullable = true)
 |-- number: integer (nullable = true)
 |-- grid: integer (nullable = true)
 |-- position: integer (nullable = true)
 |-- position_text: string (nullable = true)
 |-- position_order: integer (nullable = true)
 |-- points: double (nullable = true)
 |-- laps: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- milliseconds: integer (nullable = true)
 |-- fastest_lap: integer (nullable = true)
 |-- ranks: integer (nullable = true)
 |-- fastest_lap_time: string (nullable = true)
 |-- fastest_lap_speed: double (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)

